In [ ]:
# The goal of this notebook is to explore how we can infer a PV's metadata (orientation, tilt, area)
# from its power output over some time.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import datetime
import os
import pathlib

import numpy as np
import pandas as pd

import altair as alt
alt.data_transformers.disable_max_rows()

import plotly.express as px

from psp.data import *

def _(df, *args, **kwargs):
    print(len(df))
    display(df.head(*args, **kwargs))
    
def lvl(df, name):
    return df.index.get_level_values(name)

In [ ]:
# It's always annoying to set the working directory: we use an environment variable defined in the Makefile.
CWD = os.environ.get("CWD")
if CWD:
    os.chdir(CWD)

In [ ]:
%pwd

In [ ]:
file = 'data/5min_3/5min_all.parquet'
#file = 'data/5min/5min_100_10k.parquet'
df = pd.read_parquet(file)#[[C.power]]#[[C.power, C.date, C.id]]
_(df)

In [ ]:
df = df[[C.power]]

In [ ]:
# Filtering, mostly to run faster
# df = df[df[C.date].dt.year == 2019]

In [ ]:
# df_idx = df.set_index([C.id, C.date], drop=False).sort_index()
# _(df_idx)
df_idx = df

In [ ]:
meta = pd.read_csv('data/metadata_sensitive.csv')
meta = meta.drop(columns='Unnamed: 0')
_(meta)

In [ ]:
meta_idx = meta.set_index(C.id, drop=False)
_(meta_idx)

In [ ]:
max_power = get_max_power_for_time_of_day(df_idx[[C.power]], radius=7, min_records=10)
max_power[C.id] = lvl(max_power, C.id)
max_power[C.date] = lvl(max_power, C.date)
_(max_power)

In [ ]:
data = max_power.copy()

# Some filtering
# Keeping one day per month
data=data[data[C.date].dt.day == 1]


data = (
    data
    .reset_index(drop=True)
    .groupby(C.id)
    .rolling('1h', on=C.date, center=True, min_periods=4, closed='both').mean()
    .reset_index()
    .set_index([C.id, C.date], drop=False)
    .drop(columns='level_1')
    .sort_index()
)

# Keep note of the smooth max power.
max_power_smooth = data.copy()
_(max_power_smooth)

In [ ]:


# chart = (
#     alt.Chart(data.reset_index(drop=True))
#     .mark_line()
#     .encode(
#         x=f'hoursminutes({C.date})',
#         y=C.power,
#         column=alt.Column(f'yearmonthdate({C.date})')
#     ).properties(
#         width=100, height=100,
#     )
# )
# display(chart)

In [ ]:
from psp.pv import get_irradiance

new_meta = pd.read_csv('new_meta.csv')
new_meta = meta.join(new_meta.set_index(C.id), on=C.id, rsuffix='_infered')
new_meta = new_meta[~new_meta['capacity'].isnull()]
new_meta['diff'] = abs(new_meta['orientation'] - new_meta['orientation_infered'])
new_meta = new_meta.sort_values('diff', ascending=False)
new_meta = new_meta.set_index(C.id)
# _(new_meta)


#x = max_power.copy()
#x = x[x[C.date].dt.day == 1]

for idx in range(len(new_meta)):
    ss_id = new_meta.index[idx]
    print(f'ss_id: {ss_id}')
    
    data = df.loc[[ss_id]]
    data = get_max_power_for_time_of_day(data, radius=7, min_records=10)
    data[C.date] = data.index.get_level_values(1)
    
    # Some filtering to get a manageable plot.
    #data = data[data[C.date].dt.day == 1]
    #data = data[data[C.date].dt.month % 3 == 0]
    
    # Keep n dates.
    all_dates = data[C.date]
    dates = all_dates[::len(all_dates) // 10].dt.date
    
    data = data[data[C.date].dt.date.isin(dates)]
    
#     _(data)
    
    # Get the `dates` as needed by `get_irradiance`.
    dates = lvl(data, C.date)
    
    row = meta_idx.loc[ss_id]

    lat, lon = row[C.lat], row[C.lon]

    tilt0 = row['tilt']
    orientation0 = row['orientation']

    tilt1 = new_meta.loc[ss_id]['tilt_infered']
    orientation1 = new_meta.loc[ss_id]['orientation_infered']

    import warnings

    for tilt, orientation in [
#         [tilt0, orientation0],
        [tilt1, orientation1]
    ]:
        print(f'tilt: {tilt}')
        print(f'orientation: {orientation}')
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            irr = get_irradiance((lat, lon), dates, tilt, orientation)[['poa_global']]
            irr /= irr.max()
    #         _(data[C.power], 100)
            irr['data'] = data[C.power].tolist()
            irr['data'] /= irr['data'].quantile(0.99)
    #         _(irr, 100)
            irr = irr.stack().reset_index(level=1).rename(columns={'level_1': 'measure', 0: 'value'})

            chart = (
                alt.Chart(irr.reset_index(names=C.date))
                .mark_line()
                .encode(
                    x=f'hoursminutes({C.date})',
                    y='value',
                    column=alt.Column(f'yearmonthdate({C.date})'),
                    color='measure'
                ).properties(
                    width=100, height=100,
                )
            )
            display(chart)


        # Get irradiance data for summer and winter solstice, assuming 25 degree tilt
        # and a south facing array
        # summer_irradiance = get_irradiance(site, '06-20-2020', 25, 180)
        # winter_irradiance = get_irradiance(site, '12-21-2020', 25, 180)

In [ ]:
import scipy
from tqdm.auto import tqdm
from psp.pv import get_irradiance
from psp.optimisation import grid_bisect_optimise


data = max_power_smooth.copy()

ss_ids = data[C.id].unique().tolist()

rows = []

i = 0
#print(len(ss_ids))
for ss_id in tqdm(ss_ids):
#     print(ss_id)
    
    ref = data[(
        data[C.date].dt.day == 1
    ) & (
        data[C.id] == ss_id
    )][C.power].copy()
    

    dates = lvl(ref, C.date)
    meta_row = meta_idx.loc[ss_id]
    lat, lon = meta_row[C.lat], meta_row[C.lon]
    kwp = meta_row['kwp']
    
    cap = ref.quantile(0.99)
    
    rows.append({'cap' :cap, 'kwp': kwp})
    continue

    def compute_match(tilt, orientation):
        irr = get_irradiance((lat, lon), dates, tilt=tilt, orientation=orientation)
        values = irr['poa_global']

        values = values / values.mean()
        ref2 = ref / ref.mean()

        return - ((values - ref2) ** 2).mean()
    
    def compute_match2(params):
        return -compute_match(*params)
    
    print('custom')
    params = grid_bisect_optimise(
        compute_match,
        # Start at tilt=1 to have a different value for the different orientations.
        # This helps our naive approach to converge.
        dict(tilt=[5, 60], orientation=[5, 355]),
        precision=1,
        verbose=False,
        max_iter=100
    )
    print(params)
    
    print('scipy')
    result = b
    t, o = result.x
    print({'tilt': t, 'orientation': o})
    
    
    
    # params = grid_bisect_optimise(compute_match, dict(orientation=[0, 360]), precision=5, verbose=True)
    #print(params)
    
    rows.append({
        'ss_id': ss_id,
        'original_orientation': meta_row['orientation'],
        'original_tilt': meta_row['tilt'],
        'infered_orientation': params['orientation'],
        'infered_tilt': params['tilt'],
    })
    i += 1


    
df2 = pd.DataFrame.from_records(rows)
_(df2)

In [ ]:
df

In [ ]:
display(alt.Chart(df2).mark_point().encode(
    x='kwp',
    y='cap',
))

display(alt.Chart(df2[df2['kwp'] != 4.]).mark_point().encode(
    x='kwp',
    y='cap',
))

In [ ]:
for which in ['orientation', 'tilt']:
    data = df[['infered_' + which, 'original_' + which]]
    chart = (
        alt.Chart(data)
        .mark_point()
        .encode(
            x=alt.X('original_' + which),
            y=alt.Y('infered_' + which),
        )
    )
    max_ = data.max().max()
    line = (
        alt.Chart(pd.DataFrame(dict(x=[0, max_], y=[0, max_])))
        .mark_line(color='black')
        .encode(x='x', y='y')
    )
    display(chart + line)

In [ ]:
df['diff'] = abs(df['original_orientation'] - df['infered_orientation'])

In [ ]:
df = df.sort_values('diff', ascending=False)
df.head(20)

In [ ]:
df.describe()

In [ ]:
# Test the results after having run a separate script to compute the angles
new_meta = pd.read_csv('new_meta.csv')
new_meta = meta.join(new_meta.set_index(C.id), on=C.id, rsuffix='_infered')
new_meta = new_meta[~new_meta['capacity'].isnull()]
_(new_meta)

In [ ]:
for x, y in [
 ['orientation', 'orientation_infered'],
 ['tilt', 'tilt_infered'],
 ['kwp', 'capacity']]:

    data = new_meta.copy()

    chart = alt.chart = (
            alt.Chart(data)
            .mark_point()
            .encode(
                x=x,
                y=y
            )
        )
    max_ = data[[x, y]].max().max()
    line = (
        alt.Chart(pd.DataFrame(dict(x=[0, max_], y=[0, max_])))
        .mark_line(color='black')
        .encode(x='x', y='y')
    )
    display(chart + line)

In [ ]:
new_meta['diff_orientation'] = abs(new_meta['orientation'] - new_meta['orientation_infered'])
_(new_meta.sort_values('diff_orientation', ascending=False), 20)

In [ ]:
idx=0

new_meta.sort_values('diff_orientation', ascending=False)[idx]
data = max_power.copy()